## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,General Luna,PH,13.6881,122.1708,75.56,90,87,2.15,overcast clouds
1,1,Souillac,MU,-20.5167,57.5167,80.98,74,75,4.61,broken clouds
2,2,Longyearbyen,SJ,78.2186,15.6401,12.04,67,40,13.80,scattered clouds
3,3,Cape Town,ZA,-33.9258,18.4232,78.12,46,0,29.93,clear sky
4,4,Berezovka,BY,53.7214,25.4993,32.43,97,100,17.20,light snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,General Luna,PH,13.6881,122.1708,75.56,90,87,2.15,overcast clouds
1,1,Souillac,MU,-20.5167,57.5167,80.98,74,75,4.61,broken clouds
3,3,Cape Town,ZA,-33.9258,18.4232,78.12,46,0,29.93,clear sky
9,9,Amapa,BR,1.0000,-52.0000,78.15,80,100,3.74,overcast clouds
12,12,Bengkulu,ID,-3.8004,102.2655,78.21,81,81,3.47,broken clouds
13,13,Hermanus,ZA,-34.4187,19.2345,77.90,61,3,13.58,clear sky
14,14,Atuona,PF,-9.8000,-139.0333,77.16,76,3,14.88,clear sky
21,21,Saint-Philippe,RE,-21.3585,55.7679,80.19,80,56,10.20,broken clouds
24,24,Taoudenni,ML,22.6783,-3.9836,83.61,10,90,13.73,overcast clouds
25,25,Tacuarembo,UY,-31.7333,-55.9833,78.76,34,0,9.69,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                182
City                   182
Country                181
Lat                    182
Lng                    182
Max Temp               182
Humidity               182
Cloudiness             182
Wind Speed             182
Current Description    182
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,General Luna,PH,75.56,overcast clouds,13.6881,122.1708,
1,Souillac,MU,80.98,broken clouds,-20.5167,57.5167,
3,Cape Town,ZA,78.12,clear sky,-33.9258,18.4232,
9,Amapa,BR,78.15,overcast clouds,1.0000,-52.0000,
12,Bengkulu,ID,78.21,broken clouds,-3.8004,102.2655,
13,Hermanus,ZA,77.90,clear sky,-34.4187,19.2345,
14,Atuona,PF,77.16,clear sky,-9.8000,-139.0333,
21,Saint-Philippe,RE,80.19,broken clouds,-21.3585,55.7679,
24,Taoudenni,ML,83.61,overcast clouds,22.6783,-3.9836,
25,Tacuarembo,UY,78.76,clear sky,-31.7333,-55.9833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
          "radius": 5000,
          "type": "lodging",
          "key": g_key
         }

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,General Luna,PH,75.56,overcast clouds,13.6881,122.1708,Mamao Beach Resort
1,Souillac,MU,80.98,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
3,Cape Town,ZA,78.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
9,Amapa,BR,78.15,overcast clouds,1.0000,-52.0000,
12,Bengkulu,ID,78.21,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
13,Hermanus,ZA,77.90,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
14,Atuona,PF,77.16,clear sky,-9.8000,-139.0333,Villa Enata
21,Saint-Philippe,RE,80.19,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
24,Taoudenni,ML,83.61,overcast clouds,22.6783,-3.9836,
25,Tacuarembo,UY,78.76,clear sky,-31.7333,-55.9833,Complejo San Fructuoso


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace = True)
hotel_df.dropna(subset = ['Hotel Name'], inplace = True)
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,General Luna,PH,75.56,overcast clouds,13.6881,122.1708,Mamao Beach Resort
1,Souillac,MU,80.98,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
3,Cape Town,ZA,78.12,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
12,Bengkulu,ID,78.21,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
13,Hermanus,ZA,77.90,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
...,...,...,...,...,...,...,...
669,Praya,CV,75.74,broken clouds,14.9215,-23.5087,LT Aparthotel
670,San Andres,CO,82.42,few clouds,12.5847,-81.7006,Hotel Arena Blanca
680,Saint-Joseph,RE,79.18,scattered clouds,-21.3667,55.6167,"""Plantation Bed and Breakfast"
686,Progreso,MX,77.11,scattered clouds,21.2833,-89.6667,Playa Linda Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))